## imports

In [1]:
import os
import sys
sys.path.append('../')
from glob import glob
import torch
import numpy as np
import pandas as pd

## get and split data

In [2]:
from apulu.dataset import DatasetGenerator

data_root = '../data'

sotck_path = os.path.join(
        data_root, 'raw', 'stock', 'raw.csv'
)

sec_path = os.path.join(
        data_root, 'raw', 'sec'
)

output_path = os.path.join(
        data_root, 'processed'
)

data_list = sorted(glob(
    os.path.join(
        data_root, 'raw', 'news', '*q*.npy'
    )
))
dg = DatasetGenerator(
    data_list = data_list,
    stock_path=sotck_path,
    sec_path=sec_path,
    freq='quarter'
)

## model definition

In [3]:
from apulu.model import GCN

## model training

In [4]:
import argparse
from torch_geometric_temporal.signal import temporal_signal_split
from apulu.utils import Trainer

dataset = dg.process()
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)


INPUT_SHAPE = next(iter(train_dataset)).x.shape[1]
model = GCN(input_size = INPUT_SHAPE, hidden_dims=64)
args = argparse.Namespace(
    num_epochs = 500,
    learning_rate = 1e-3,
    device = "cpu",
    val_size = .1,
    verbose = False
)

In [5]:
trainer = Trainer(model, train_dataset, args, test_dataset)

In [6]:
model = trainer.train()

100%|███████████████████████████████████████████████████████████████████████████| 500/500 [00:09<00:00, 52.25it/s]


            best model loss is:
                val loss: 0.5018613785505295 @ epoch: 408
            
==================validation set performance==================
ROC AUC score 0.7987878787878788
              precision    recall  f1-score   support

           0       0.72      0.84      0.78        25
           1       0.86      0.76      0.81        33

    accuracy                           0.79        58
   macro avg       0.79      0.80      0.79        58
weighted avg       0.80      0.79      0.79        58

==================test set performance==================
ROC AUC score 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        30
           1       0.74      1.00      0.85        86

    accuracy                           0.74       116
   macro avg       0.37      0.50      0.43       116
weighted avg       0.55      0.74      0.63       116




/home/jiekie/anaconda3/envs/apulu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jiekie/anaconda3/envs/apulu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jiekie/anaconda3/envs/apulu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result